In [1]:
import ROOT
import numpy as np

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8bb15c0


In [2]:
n = 2
n_bins = 7
n_events_data = 500000
n_events_mc = n_events_data * 10
val_theta = 0.5
delta_theta = 0.01

x = []
for i in range(n):
    var = ROOT.RooRealVar(f"x_{i}", f"x_{i}", 0., -3., 3.)
    var.setBins(n_bins)
    x.append(var)

In [3]:
cov_mat1 = ROOT.TMatrixD(ROOT.TMatrixD.kUnit, ROOT.TMatrixD(n, n))
cov_mat1[0][1] = 0.42
cov_mat1[1][0] = 0.42
gauss1 = ROOT.RooMultiVarGaussian("gauss1", "gauss1", ROOT.RooArgList(x), cov_mat1)

cov_mat2 = ROOT.TMatrixD(ROOT.TMatrixD.kUnit, ROOT.TMatrixD(n, n))
cov_mat2[0][1] = 0.1337
cov_mat2[1][0] = 0.1337
gauss2 = ROOT.RooMultiVarGaussian("gauss2", "gauss2", ROOT.RooArgList(x), cov_mat2)

In [4]:
# our mixing angle
theta = ROOT.RooRealVar("theta", "theta", val_theta, 0., 1.)

gen_model = ROOT.RooAddPdf("gen_model", "gen_model", gauss1, gauss2, theta)

In [5]:
gen_model.Print("t")

0xb4f9d00 RooAddPdf::gen_model = 1/1 [Auto,Clean] 
  0xb38baa0/V- RooMultiVarGaussian::gauss1 = 1 [Auto,Dirty] 
    0x8b0ee10/V- RooRealVar::x_0 = 0
    0xa934470/V- RooRealVar::x_1 = 0
    0xb3ba580/V- RooConstVar::0 = 0
  0xb429620/V- RooMultiVarGaussian::gauss2 = 1 [Auto,Dirty] 
    0x8b0ee10/V- RooRealVar::x_0 = 0
    0xa934470/V- RooRealVar::x_1 = 0
    0xb3ba580/V- RooConstVar::0 = 0
  0xaf71c70/V- RooRealVar::theta = 0.5


In [6]:
# data = gen_model.generate(x, 1)
data_unbinned = gen_model.generate(x, n_events_data)
template_unbinned = gen_model.generate(x, n_events_mc)
data = data_unbinned.binnedClone()
data.Print("v")

DataStore gen_modelData_binned (Generated From gen_model_binned)
  Contains 49 entries
  Observables: 
    1)  x_0 = 2.57143  L(-3 - 3) B(7)  "x_0"
    2)  x_1 = 2.57143  L(-3 - 3) B(7)  "x_1"
Binned Dataset gen_modelData_binned (Generated From gen_model_binned)
  Contains 49 bins with a total weight of 500000
  Observables:     1)  x_0 = 2.57143  L(-3 - 3) B(7)  "x_0"
    2)  x_1 = 2.57143  L(-3 - 3) B(7)  "x_1"


In [7]:
# plot generated data
frame0 = x[0].frame()
frame1 = x[1].frame()

In [8]:
data.plotOn(frame0)
gen_model.plotOn(frame0)
data.plotOn(frame1)
gen_model.plotOn(frame1)

[#1] INFO:Plotting -- RooAbsReal::plotOn(gen_model) plot on x_0 integrates over variables (x_1)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(gen_model_Int[x_1]_Norm[x_0,x_1]) using numeric integrator RooIntegrator1D to calculate Int(x_1)
[#1] INFO:Plotting -- RooAbsReal::plotOn(gen_model) plot on x_1 integrates over variables (x_0)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(gen_model_Int[x_0]_Norm[x_0,x_1]) using numeric integrator RooIntegrator1D to calculate Int(x_0)


In [9]:
c0 = ROOT.TCanvas()
frame0.Draw()
c0.Draw()

c1 = ROOT.TCanvas()
frame1.Draw()
c1.Draw()

In [10]:
# nll = gen_model.createNLL(data)

In [11]:
# nll_minimizer = ROOT.RooMinimizer(nll)

In [12]:
# obviously cannot fit theta as the relevant information is lost in the projection
# and there is no element to recover it added to the fit
# I could space the gaussians a bit but then it becomes very trivial?
# nll_minimizer.migrad()

In [13]:
# frame_theta = theta.frame()
# nll.plotOn(frame_theta, ShiftToZero=True)
# c_theta = ROOT.TCanvas()
# frame_theta.Draw()
# c_theta.Draw()

In [14]:
# naive fit model 1
# make 1d histos from template
gauss1_tmpl = gauss1.generateBinned(x, n_events_mc)
gauss2_tmpl = gauss2.generateBinned(x, n_events_mc)

gauss1_tmpl_0 = gauss1_tmpl.reduce(x[0])
gauss1_tmpl_1 = gauss1_tmpl.reduce(x[1])
gauss2_tmpl_0 = gauss2_tmpl.reduce(x[0])
gauss2_tmpl_1 = gauss2_tmpl.reduce(x[1])

In [15]:
gauss1_tmpl_0_hf = ROOT.RooHistFunc("gauss1_tmpl_0_hf", "gauss1_tmpl_0_hf", x[0], gauss1_tmpl_0)
gauss1_tmpl_1_hf = ROOT.RooHistFunc("gauss1_tmpl_1_hf", "gauss1_tmpl_1_hf", x[1], gauss1_tmpl_1)
gauss2_tmpl_0_hf = ROOT.RooHistFunc("gauss2_tmpl_0_hf", "gauss2_tmpl_0_hf", x[0], gauss2_tmpl_0)
gauss2_tmpl_1_hf = ROOT.RooHistFunc("gauss2_tmpl_1_hf", "gauss2_tmpl_1_hf", x[1], gauss2_tmpl_1)

In [16]:
fit_theta = ROOT.RooRealVar("fit_theta", "fit_theta", val_theta, 0., 1.)

sum_tmpl_0 = ROOT.RooRealSumPdf("sum_tmpl_0", "sum_tmpl_0", gauss1_tmpl_0_hf, gauss2_tmpl_0_hf, fit_theta)
sum_tmpl_1 = ROOT.RooRealSumPdf("sum_tmpl_1", "sum_tmpl_1", gauss1_tmpl_1_hf, gauss2_tmpl_1_hf, fit_theta)

In [17]:
fit_model = ROOT.RooProdPdf("fit_model", "fit_model", sum_tmpl_0, sum_tmpl_1)
fit_model.Print("t")

0xc7d7700 RooProdPdf::fit_model = 2.93864e+12 [Auto,Dirty] 
RooProdPdf begin partial integral cache
[0]0xc71a2d0 RooRealSumPdf::sum_tmpl_0 = 1.71368e+06 [Auto,Dirty] 
[0]  0xc6546a0/V- RooHistFunc::gauss1_tmpl_0_hf = 1.71464e+06 [Auto,Clean] 
[0]    0x8b0ee10/V- RooRealVar::x_0 = 0
[0]  0x6f47bd0/V- RooHistFunc::gauss2_tmpl_0_hf = 1.71274e+06 [Auto,Clean] 
[0]    0x8b0ee10/V- RooRealVar::x_0 = 0
[0]  0x6f4a090/V- RooRealVar::fit_theta = 0.5
[0]0xc7144a0 RooRealSumPdf::sum_tmpl_1 = 1.71481e+06 [Auto,Dirty] 
[0]  0x6f6f710/V- RooHistFunc::gauss1_tmpl_1_hf = 1.71549e+06 [Auto,Clean] 
[0]    0xa934470/V- RooRealVar::x_1 = 0
[0]  0xc643b00/V- RooHistFunc::gauss2_tmpl_1_hf = 1.71412e+06 [Auto,Clean] 
[0]    0xa934470/V- RooRealVar::x_1 = 0
[0]  0x6f4a090/V- RooRealVar::fit_theta = 0.5
RooProdPdf end partial integral cache
  0xc71a2d0/V- RooRealSumPdf::sum_tmpl_0 = 1.71368e+06 [Auto,Dirty] 
    0xc6546a0/V- RooHistFunc::gauss1_tmpl_0_hf = 1.71464e+06 [Auto,Clean] 
      0x8b0ee10/V- RooRealVa

In [18]:
fit_model.fitTo(data)

<cppyy.gbl.RooFitResult object at 0x(nil)>

[#1] INFO:Fitting -- RooAbsPdf::fitTo(fit_model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 2.17394 ms
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_fit_model_gen_modelData_binned) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 1431871.91116576758
Edm   = 2.32745076385238706e-07
Nfcn  = 50
fit_theta	  = 3.55338e-09	 +/-  0.0076345	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 135.4 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       1431905.554 Edm =       318.4449368 NCalls =      5
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 1431905.554
  Edm           : 318.4449368
  Internal parameters:	[                0]	
  Internal gradient  :	[      33.15634795]	
  Internal covariance matrix:
[[      1.1586732]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       1431905.554 Edm =       318.4449368 NCalls =      5
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       1431877.358 Edm =        95.3210623 NCalls =     19
Info in <Minuit2>: VariableMetricBuilder    2 - FCN =       1431875.745 Edm =       266.1239619 NCalls =     31
Info in <Minuit2>: VariableMetricBui

In [19]:
# naive fit model 2: optimal observable
S_0 = ROOT.RooAddPdf("gen_model", "gen_model", gauss1, gauss2, ROOT.RooFit.RooConst(val_theta))
S_1 = gauss2
O = ROOT.RooRatio("oo", "oo", S_1, S_0)
O.Print("t")

0xcb0b800 RooRatio::oo = [#0] WARNING:Eval -- Evaluating RooAddPdf gen_model without a defined normalization set. This can lead to ambiguous coefficients definition and incorrect results. Use RooAddPdf::fixCoefNormalization(nset) to provide a normalization set for defining uniquely RooAddPdf coefficients!
1 [Auto,Dirty] 
  0xb429620/V- RooMultiVarGaussian::gauss2 = 1 [Auto,Dirty] 
    0x8b0ee10/V- RooRealVar::x_0 = 0
    0xa934470/V- RooRealVar::x_1 = 0
    0xb3ba580/V- RooConstVar::0 = 0
  0xcb16780/V- RooAddPdf::gen_model = 1/1 [Auto,Clean] 
    0xb38baa0/V- RooMultiVarGaussian::gauss1 = 1 [Auto,Dirty] 
      0x8b0ee10/V- RooRealVar::x_0 = 0
      0xa934470/V- RooRealVar::x_1 = 0
      0xb3ba580/V- RooConstVar::0 = 0
    0xb429620/V- RooMultiVarGaussian::gauss2 = 1 [Auto,Dirty] 
      0x8b0ee10/V- RooRealVar::x_0 = 0
      0xa934470/V- RooRealVar::x_1 = 0
      0xb3ba580/V- RooConstVar::0 = 0
    0xcab6420/V- RooConstVar::0.5 = 0.5


In [20]:
# building the template parametrisation is annoying
# we want to reweight instead of generating a second template to not have relative errors
# so we need to clone the gen_model, replace the theta server by theta+delta_theta
# make that the rw_model
# and then make obs_ds_rw with weights rw_model/gen_model

shifted_theta = ROOT.RooRealVar("shifted_theta", "shifted_theta", val_theta + delta_theta, 0., 1.)

rw_model = gen_model.cloneTree()
old_theta = rw_model.findServer("theta")
print(old_theta)
shifted_theta.setAttribute("ORIGNAME:theta", True)
rw_model.redirectServers(ROOT.RooArgList([shifted_theta]), False, True)
rw_model.Print("t")

RooRealVar::theta = 0.5  L(0 - 1) 

0xcb03840 RooAddPdf::gen_model = 1/1 [Auto,Clean] 
  0xcae6e10/V- RooMultiVarGaussian::gauss1 = 1 [Auto,Dirty] 
    0xc847850/V- RooRealVar::x_0 = 0
    0xc88c210/V- RooRealVar::x_1 = 0
    0xcb7fc30/V- RooConstVar::0 = 0
  0xcb09fa0/V- RooMultiVarGaussian::gauss2 = 1 [Auto,Dirty] 
    0xc847850/V- RooRealVar::x_0 = 0
    0xc88c210/V- RooRealVar::x_1 = 0
    0xcb7fc30/V- RooConstVar::0 = 0
  0xc836ca0/V- RooRealVar::shifted_theta = 0.51


In [21]:
# make oo dataset
O.attachDataSet(template_unbinned)
rw_model.attachDataSet(template_unbinned)
S_0.attachDataSet(template_unbinned)

x_arg_set = ROOT.RooArgSet(x)

tmp_O_ds = []
weights = []
for i in range(template_unbinned.numEntries()):
    template_unbinned.get(i)
    tmp_O_ds.append(O.getVal(x_arg_set))
    weight = rw_model.getVal(x_arg_set) / S_0.getVal(x_arg_set)
    weights.append(weight)

print(np.asarray(tmp_O_ds))

[1.23191048 1.08731175 0.91105254 ... 0.89225717 0.9687308  1.03748638]


In [22]:
O_obs = ROOT.RooRealVar("O_obs", "O_obs", 1., 0.75, 1.25)
O_obs.setBins(n_bins**2)
obs_ds = ROOT.RooDataSet.from_numpy({"O_obs": np.asarray(tmp_O_ds)}, [O_obs])
obs_rw_ds = ROOT.RooDataSet.from_numpy({"O_obs": np.asarray(tmp_O_ds), "weight": np.asarray(weights)}, [O_obs], weight_name="weight")

[#0] WARNING:DataHandling -- RooDataSet.from_numpy() Ignored 488129 out-of-range events
[#0] WARNING:DataHandling -- RooDataSet.from_numpy() Ignored 488129 out-of-range events


In [23]:
frame_oo = O_obs.frame()
obs_ds.plotOn(frame_oo)
obs_rw_ds.plotOn(frame_oo, MarkerColor=ROOT.kRed)

c_oo = ROOT.TCanvas()
frame_oo.Draw()
c_oo.Draw()

[#1] INFO:InputArguments -- RooAbsData::plotOn() INFO: dataset has non-integer weights, auto-selecting SumW2 errors instead of Poisson errors
[#1] INFO:Plotting -- RooPlot::updateFitRangeNorm: New event count of 4.51221e+06 will supersede previous event count of 4.51187e+06 for normalization of PDF projections


In [24]:
O_hist = obs_ds.createHistogram("O_hist", O_obs)
O_hist_rw = obs_rw_ds.createHistogram("O_hist_rw", O_obs)
slope_hist = O_hist_rw.Clone()
slope_hist.Add(O_hist, -1.0)
slope_hist.Divide(O_hist)
slope_hist.Scale(1 / delta_theta)
c_slope_hist = ROOT.TCanvas()
slope_hist.Draw()
c_slope_hist.Draw()

In [25]:
# what one would probably do for the classical approach/fit
# OO = obs_ds.mean(O_obs) * obs_ds.numEntries()
# C = obs_ds.sigma(O_obs)

In [26]:
O_hist_dh = ROOT.RooDataHist("O_hist_dh", "O_hist_dh", [O_obs], O_hist)
slope_hist_dh = ROOT.RooDataHist("slope_hist_dh", "slope_hist_dh", [O_obs], slope_hist)

In [27]:
# prepare dataset to fit
O.attachDataSet(data_unbinned)
tmp_O_fit_ds = []
for i in range(data_unbinned.numEntries()):
    data_unbinned.get(i)
    tmp_O_fit_ds.append(O.getVal(x_arg_set))

In [28]:
O_fit_ds = ROOT.RooDataSet.from_numpy({"O_obs": np.asarray(tmp_O_fit_ds)}, [O_obs])
O_fit_dh = O_fit_ds.binnedClone()

[#0] WARNING:DataHandling -- RooDataSet.from_numpy() Ignored 48851 out-of-range events


In [ ]:
# do a simple ML template fit
sm_template = ROOT.RooHistFunc("sm_template", "sm_template", O_obs, O_hist_dh)
theta_param = ROOT.RooHistFunc("theta_param", "theta_param", O_obs, slope_hist_dh)
delta_theta_fit = ROOT.RooRealVar("dt_fit", "dt_fit", 0., -0.5, 0.5)
rel_change = ROOT.RooProduct("rel_change", "rel_change", [delta_theta_fit, theta_param])
abs_change = ROOT.RooAddition("abs_change", "abs_change", [ROOT.RooFit.RooConst(1.0), rel_change])
some_factor = ROOT.RooRealVar("some_factor", "some_factor", n_events_data, 0., 2* n_events_data)
proto_model2 = ROOT.RooProduct("proto_model2", "proto_model2", [sm_template, abs_change])
# fit_model2 = ROOT.RooRealSumPdf("fit_model2", "fit_model2", [proto_model2], [some_factor], True)
fit_model2 = ROOT.RooRealSumPdf("fit_model2", "fit_model2", [proto_model2], [])
fit_model2.Print("t")

0x10046720 RooRealSumPdf::fit_model2 = 159366 [Auto,Dirty] 
  0xfff3f30/V- RooProduct::proto_model2 = 159366 [Auto,Clean] 
    0x10073000/V- RooHistFunc::sm_template = 159366 [Auto,Clean] 
      0xcb25980/V- RooRealVar::O_obs = 1
    0x10011760/V- RooAddition::abs_change = 1 [Auto,Clean] 
      0x1005ff40/V- RooConstVar::1 = 1
      0x1002f710/V- RooProduct::rel_change = 9.80447e-07 [Auto,Clean] 
        0xfec87a0/V- RooRealVar::dt_fit = 0.1
        0xfec7ac0/V- RooHistFunc::theta_param = 9.80447e-06 [Auto,Clean] 
          0xcb25980/V- RooRealVar::O_obs = 1


In [30]:
fit_model2.fitTo(O_fit_dh)

<cppyy.gbl.RooFitResult object at 0x(nil)>

[#1] INFO:Fitting -- RooAbsPdf::fitTo(fit_model2_over_fit_model2_Int[O_obs]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- Creation of NLL object took 462.985 μs
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_fit_model2_over_fit_model2_Int[O_obs]__binned) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 500 convergence for edm < 1 strategy 1
[#1] INFO:NumericIntegration -- RooRealIntegral::init(proto_model2_Int[O_obs]) using numeric integrator RooBinIntegrator to calculate Int(O_obs)
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = -473768.169864930795
Edm   = 0.0001960956375736231
Nfcn  = 16
dt_fit	  = -0.00877856	 +/-  0.00824613	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 284.474 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -473680.9235 Edm =       91.25084166 NCalls =      5
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : -473680.9235
  Edm           : 91.25084166
  Internal parameters:	[     0.2013579208]	
  Internal gradient  :	[      784.8703735]	
  Internal covariance matrix:
[[  0.00059251725]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =      -473680.9235 Edm =       91.25084166 NCalls =      5
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       -473767.846 Edm =      0.3526252659 NCalls =      8
Info in <Minuit2>: VariableMetricBuilder    2 - FCN =      -473768.1699 Edm =   0.0002014800752 NCalls =     11
Info in <Minuit2>: MnHesse Done a